In [21]:
import os

In [22]:
BIGQUERY_CREDENTIALS_FILE_PATH = r"/Users/taylorlin/Desktop/GCP_Key/harry_GCS_BigQuery_write_cred.json"
# BIGQUERY_CREDENTIALS_FILE_PATH = r"D:\data_engineer\TIR_group2\TIR101_Group2\secrets\harry_GCS_BigQuery_write_cred.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = BIGQUERY_CREDENTIALS_FILE_PATH

In [23]:
from google.cloud import bigquery
BQ_CLIENT = bigquery.Client()  
# showing project name(level higher than dataset)
BQ_CLIENT.project

'tir101-group2-422603'

In [ ]:
# Dataset Creation
def create_dataset(dataset_name:str , location:str="asia-east1",client: bigquery.Client = BQ_CLIENT):
    """Create a dataset"""
    porject_name = client.project
    dataset_id = f"{porject_name}.{dataset_name}"

    dataset = bigquery.Dataset(dataset_id)
    dataset.location = location

    client.create_dataset(dataset)
    print(f"created {dataset_name} in project {porject_name} ,location:{location}")

create_dataset(dataset_name="ubike_history_GCS_to_BQ_SRC_ODS_DIM")

# SRC Creation

rent_time,rent_station,return_time,return_station,rent,infodate


法一、有 partition

In [58]:
def GCS_ubike_history_to_BQ_SRC(client: bigquery.Client = BQ_CLIENT) -> None:
    """create external table (GCS to BQ) for ubike_histroy data"""
    query_job = client.query(
    """
        CREATE OR REPLACE EXTERNAL TABLE `ubike_history_GCS_to_BQ_SRC_ODS_DIM.SRC_ubike_v2_history`
        (
            rent_time STRING,
            rent_station STRING,
            return_time STRING,
            return_station STRING,
            rent STRING,
            infodate STRING
        )
        WITH PARTITION COLUMNS
        OPTIONS (
            format = 'CSV',
            uris = [
                'gs://static_reference/bike_history/*.csv'
            ],
            hive_partition_uri_prefix = 'gs://static_reference/bike_history/',
            skip_leading_rows = 1,
            max_bad_records = 100,
            ignore_unknown_values = true
        )    
    """
    )
    query_job.result()
    print("SRC_ubike_v2_history is created.")
GCS_ubike_history_to_BQ_SRC()

SRC_ubike_v2_history is created.


感覺有很多無法解讀的數值

改掉
            skip_leading_rows = 1,
            max_bad_records = 100,
            ignore_unknown_values = true

改成
            skip_leading_rows = 1,
            max_bad_records = 1

# 並且改數據的型態就不行


In [59]:
def GCS_ubike_history_to_BQ_SRC(client: bigquery.Client = BQ_CLIENT) -> None:
    """create external table (GCS to BQ) for ubike_histroy data"""
    query_job = client.query(
    """
        CREATE OR REPLACE EXTERNAL TABLE `ubike_history_GCS_to_BQ_SRC_ODS_DIM.SRC_ubike_v3_history`
        (
            rent_time TIMESTAMP,
            rent_station STRING,
            return_time TIMESTAMP,
            return_station STRING,
            rent TIME,
            infodate DATE
        )
        WITH PARTITION COLUMNS
        OPTIONS (
            format = 'CSV',
        uris = [
                'gs://static_reference/bike_history/*.csv'
            ],
            hive_partition_uri_prefix = 'gs://static_reference/bike_history/',
            skip_leading_rows = 1,
            max_bad_records = 1
        )
    """
    )
    query_job.result()
    print("SRC_ubike_v3_history is created.")
GCS_ubike_history_to_BQ_SRC()

SRC_ubike_v3_history is created.


CREATE TABLE `project_id.dataset_id.bike_usage_history` (
  id INT64,
  lend_date TIME,
  lend_hour INT64,
  lend_station_id INT64,
  return_date DATE,
  return_hour INT64,
  return_station_id INT64,
  usage_time TIME,
  source_date DATE,
  create_time TIMESTAMP,
  PRIMARY KEY (id)
);

In [43]:
from google.cloud import bigquery

# 确保定义了 BQ_CLIENT 作为 bigquery.Client 的实例
BQ_CLIENT = bigquery.Client()

def GCS_bike_history_to_BQ_SRC(client: bigquery.Client = BQ_CLIENT) -> None:
    """Create external table (GCS to BQ) for ubike_history data"""
    schema = [
        bigquery.SchemaField("no", "INT64", mode="NULLABLE"),
        bigquery.SchemaField("rent_time", "TIMESTAMP", mode="NULLABLE"),
        bigquery.SchemaField("rent_station", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("return_time", "TIMESTAMP", mode="NULLABLE"),
        bigquery.SchemaField("return_station", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("rent", "TIME", mode="NULLABLE"),
        bigquery.SchemaField("infodate", "DATE", mode="NULLABLE")
    ]

    external_config = bigquery.ExternalConfig('CSV')
    external_config.source_uris = ['gs://static_reference/bike_history/*.csv']
    external_config.options.skip_leading_rows = 1
    external_config.options.max_bad_records = 10
    external_config.options.ignore_unknown_values = True

    hive_partitioning = bigquery.external_config.HivePartitioningOptions()
    hive_partitioning.source_uri_prefix = 'gs://static_reference/bike_history/'
    external_config.hive_partitioning_options = hive_partitioning

    table_id = 'tir101-group2-422603.ubike_history_GCS_to_BQ_SRC_ODS_DIM.SRC_bike_history'
    table = bigquery.Table(table_id, schema=schema)
    table.external_data_configuration = external_config

    client.create_table(table, exists_ok=True)
    print("SRC_bike_history is created.")

# 调用函数创建外部表
GCS_bike_history_to_BQ_SRC(client=BQ_CLIENT)


SRC_bike_history is created.


In [47]:
from google.cloud import bigquery

# 确保定义了 BQ_CLIENT 作为 bigquery.Client 的实例
BQ_CLIENT = bigquery.Client()

def GCS_ubike_history_to_BQ_SRC(client: bigquery.Client = BQ_CLIENT) -> None:
    """Create external table (GCS to BQ) for ubike_history data"""
    schema = [
        bigquery.SchemaField("id", "INT64", mode="NULLABLE"),
        bigquery.SchemaField("rent_time", "TIMESTAMP", mode="NULLABLE"),
        bigquery.SchemaField("rent_station", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("return_time", "TIMESTAMP", mode="NULLABLE"),
        bigquery.SchemaField("return_station", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("rent", "TIME", mode="NULLABLE"),
        bigquery.SchemaField("infodate", "DATE", mode="NULLABLE")
    ]

    external_config = bigquery.ExternalConfig('CSV')
    external_config.source_uris = ['gs://static_reference/bike_history/*.csv']
    external_config.options.skip_leading_rows = 1
    external_config.options.max_bad_records = 10
    external_config.options.ignore_unknown_values = True

    hive_partitioning = bigquery.external_config.HivePartitioningOptions()
    hive_partitioning.source_uri_prefix = 'gs://static_reference/bike_history/'
    hive_partitioning.require_partition_filter = True
    hive_partitioning.mode = 'AUTO'
    external_config.hive_partitioning_options = hive_partitioning

    table_id = 'tir101-group2-422603.ubike_history_GCS_to_BQ_SRC_ODS_DIM.SRC_bike_history'
    table = bigquery.Table(table_id, schema=schema)
    table.external_data_configuration = external_config

    client.create_table(table, exists_ok=True)
    print("SRC_bike_history is created.")

# 调用函数创建外部表
GCS_ubike_history_to_BQ_SRC(client=BQ_CLIENT)


SRC_bike_history is created.


這個成功了

In [50]:
from google.cloud import bigquery
from google.cloud.exceptions import NotFound

# 确保定义了 BQ_CLIENT 作为 bigquery.Client 的实例
BQ_CLIENT = bigquery.Client()

def GCS_ubike_history_to_BQ_SRC(client: bigquery.Client = BQ_CLIENT) -> None:
    """Create or replace external table (GCS to BQ) for ubike_history data"""
    table_id = 'tir101-group2-422603.ubike_history_GCS_to_BQ_SRC_ODS_DIM.SRC_bike_history'

    # 尝试删除已存在的表
    try:
        client.delete_table(table_id)  # 如果表已存在则删除
        print(f"Table {table_id} deleted.")
    except NotFound:
        print(f"Table {table_id} does not exist.")

    schema = [
        bigquery.SchemaField("id", "INT64", mode="NULLABLE"),
        bigquery.SchemaField("rent_time", "TIMESTAMP", mode="NULLABLE"),
        bigquery.SchemaField("rent_station", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("return_time", "TIMESTAMP", mode="NULLABLE"),
        bigquery.SchemaField("return_station", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("rent", "TIME", mode="NULLABLE"),
        bigquery.SchemaField("infodate", "DATE", mode="NULLABLE")
    ]

    external_config = bigquery.ExternalConfig('CSV')
    external_config.source_uris = ['gs://static_reference/bike_history/*.csv']
    external_config.options.skip_leading_rows = 1
    external_config.options.max_bad_records = 10
    external_config.options.ignore_unknown_values = True

    hive_partitioning = bigquery.external_config.HivePartitioningOptions()
    external_config.hive_partitioning_options = hive_partitioning

    table = bigquery.Table(table_id, schema=schema)
    table.external_data_configuration = external_config

    client.create_table(table)
    print("SRC_bike_history is created.")

# 调用函数创建外部表
GCS_ubike_history_to_BQ_SRC(client=BQ_CLIENT)


Table tir101-group2-422603.ubike_history_GCS_to_BQ_SRC_ODS_DIM.SRC_bike_history does not exist.
SRC_bike_history is created.
